In [3]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.spaces import MultiDiscrete, Discrete

In [4]:
print(gym.__version__)

0.28.1


In [5]:
import random 

# Creating a Custom Environment



In [44]:
# make an environment
import random

class SchedulerEnvironment(gym.Env):
    def __init__(self, scenario_one = True, terminate_num = 200, mean_delay_one = 6, mean_delay_two = 4):
        self.scenario_one = scenario_one

        self.mean_delay_one = mean_delay_one
        self.mean_delay_two = mean_delay_two


        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 

        self.incoming_packets = dict(
            {
                0: self.queue_one_incoming_packet,
                1: self.queue_two_incoming_packet,
                2: self.queue_best_effort_incoming_packet

            }
            )


        self.step_counter = 0 # i.e. timeslots
        self.terminate_num = terminate_num

        # maximum size of the queue
        self.max_queue_size = 10
        
        # can adjust size of max delay in timeslots after initial observations
        MAX_DELAY = 100

        queue_sample = np.arange(self.max_queue_size)
        for i in range(self.max_queue_size):
            queue_sample[i] = MAX_DELAY

        self.observation_space = spaces.Dict( 
            {
                
                "queues": MultiDiscrete(queue_sample), 
                                
#                 boolean for timeslot delay during switch 
#                 zero for no delay, one for delay. 
                "switchCounter": Discrete(2)
                
            }
        )

        self.queue_one_delay = [] 

        self.queue_two_delay = []

        self.queue_best_effort_delay = [] 

        self.queue_delays = dict(
            {
                0: self.queue_one_delay,
                1: self.queue_two_delay,
                2: self.queue_best_effort_delay
            }
        )

        self.queue_one_removed = 0
        self.queue_two_removed = 0
        self.queue_best_effort_removed = 0

        self.removed_packets = dict(
            {
                0: self.queue_one_removed,
                1: self.queue_two_removed,
                2: self.queue_best_effort_removed
            }
        )

        self.current_state = None
        
        self.action_space = spaces.Discrete(3)
        
        self._action_to_queue = {
                0: self.queue_delays[0],
                1: self.queue_delays[1],
                2: self.queue_delays[2]
            }

#         assert render_mode is None or render_mode in self.metadata["render_modes"]
#         self.render_mode = render_mode
        
#         """
#         If human-rendering is used, `self.window` will be a reference
#         to the window that we draw to. `self.clock` will be a clock that is used
#         to ensure that the environment is rendered at the correct framerate in
#         human-mode. They will remain `None` until human-mode is used for the
#         first time.
#         """
#         self.window = None
#         self.clock = None

    def _increment_delay(self):
        for i in range(len(self.queue_delays)):    
            for j in range(len(self.queue_delays[i])):
                self.queue_delays[i][j] += 1

            
    # def _add_delay_to_queue(self):
    #     queues = self.observation_space["queue_delay"]
    #     # this will give the index of the queue that had the packet selected
    #     selected_queue = self.action_space.sample() 
    #     for q in range(len(queues)):
    #         if (selected_queue != q):
    #             queues[q] += 1
    #         else:
    #             queues[q] = 0

    def _can_add_packets(self, packet_arrival, action ):
        # increment before evaluating arrival. 
        self.incoming_packets[action] += packet_arrival
        packet_arrived = self.incoming_packets[action] >= 1.0
        not_max_size = len(self.queue_delays[action]) < self.max_queue_size

        if packet_arrived and not_max_size:
            self.incoming_packets[action] -= 1.0
            self.queue_delays[action].append(0)                     

    def _add_packets(self):
        
        PACKET_ARRIVAL_ONE = 0.3
        PACKET_ARRIVAL_TWO = 0.25
        PACKET_ARRIVAL_BEST_EFFORT = 0.4
        
        self._can_add_packets(PACKET_ARRIVAL_ONE, 0)
        self._can_add_packets(PACKET_ARRIVAL_TWO, 1)
        self._can_add_packets(PACKET_ARRIVAL_BEST_EFFORT, 2)

        return
     
    def _modify_states(self):

        # will not calculate delay for freshly added packets. 
        self._increment_delay()
        self._add_packets()
        return 
    
    def _get_obs(self):
        return { 
            "queues": self.queue_delays
            }
    
    def _calculate_avg_delay(self, queue):
        queue_size = len(queue)
        if (queue_size == 0):
            return 0
        
        # sum
        sum = 0
        for i in range(queue_size):
            sum += queue[i]
        
        # avg
        avg = sum / queue_size
        return avg

    def _get_info(self):

        avg_one = self._calculate_avg_delay(self.queue_delays[0])
        avg_two = self._calculate_avg_delay(self.queue_delays[1])
        avg_best = self._calculate_avg_delay(self.queue_delays[2])

        # return average delays for each queue. 
        avg_mean_all_queues = [avg_one, avg_two, avg_best]
        
        # return the number of packets removed in each queue. 
        
        packets_removed = [self.removed_packets[0], self.removed_packets[1], self.removed_packets[2]]

        # return the queue num.
        return [avg_mean_all_queues, packets_removed, self.step_counter + 1]
 
    def _initialise_delays(self, queue):
        # give a random delay between 0 and 5
        size_of_queue = len(queue)
        for i in range(size_of_queue):
            queue[i] = random.randint(0,5)

    def _set_all_zero(self):
        for i in range((3)):
            self.queue_delays[i].clear()
            self.removed_packets[i] = 0

        self.step_counter = 0
        self.current_state = None
        self.queue_one_incoming_packet = 0.0
        self.queue_two_incoming_packet = 0.0
        self.queue_best_effort_incoming_packet = 0.0 
    

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # reset everything 
        self._set_all_zero()

        # set the size of the queue. 
        self.queue_delays[0] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[1] = self.observation_space["queues"].sample().tolist()
        self.queue_delays[2] = self.observation_space["queues"].sample().tolist()

        self._initialise_delays(self.queue_delays[0])
        self._initialise_delays(self.queue_delays[1])
        self._initialise_delays(self.queue_delays[2])

        observation = self._get_obs()
        info = self._get_info()

        if self.render_mode == "human":
            self._render_frame()

        return observation, info
    
    def _reward_function(self, action, packet_delay):
        reward = 0

        mean_delays = dict(
            {
                0: self.mean_delay_one,
                1: self.mean_delay_two
            }
        )
        
        MARGIN_OF_DELAY = 1

        # determine reward (with packet_delay)
#         (mean_delays[action] > packet_delay + MARGIN_OF_DELAY)
        if (action == 2):
        
            # reward taking the best effort queue when mean delays are adjusted
            # for priority queues.
            avg_first_queue = self._calculate_avg_delay(self.queue_delays[0])
            within_margin_first_queue = ( avg_first_queue < mean_delays[0] + MARGIN_OF_DELAY) and ( avg_first_queue > mean_delays[0] - MARGIN_OF_DELAY )
            
            avg_second_queue = self._calculate_avg_delay(self.queue_delays[1])
            within_margin_second_queue = ( avg_second_queue < mean_delays[1] + MARGIN_OF_DELAY) and ( avg_second_queue > mean_delays[1] - MARGIN_OF_DELAY )
            
            if (within_margin_first_queue and within_margin_second_queue):
                reward = 1

            # Punish for not prioritising. 
            else:
                reward = -1

        else:

            # packet delay > mean delay by the margin
            if (mean_delays[action] + MARGIN_OF_DELAY < packet_delay):
                # encourage minimising the delay for this queue.
                reward = 0

            # packet delay < mean delay by the margin
            elif (mean_delays[action] - MARGIN_OF_DELAY > packet_delay):
                # discourage minimising the delay too much.
                reward = -1

            # packet delay within margin, don't perform again. 
            else:
                reward = -1

        return reward 
    
    def _retrieve_packet(self, action):
        # retrieve the packet. 
        packet_delay = self.queue_delays[action].pop(0)
        self.removed_packets[action] += 1


        return packet_delay

    def step(self, action):
        packet_delay = 0
        # reward will be -1.
        if len(self.queue_delays[action]) == 0:
            None

        else:
            if self.scenario_one:
                packet_delay = self._retrieve_packet(action)
    
            else: 
                if (action == self.current_state):
                    packet_delay = self._retrieve_packet(action)


        # perform queue switch given the conditions.
        if not self.scenario_one:
            if (action != self.current_state):
                self.current_state = action
        
        reward = self._reward_function(action, packet_delay)
                
        self._modify_states()

        info = self._get_info()

        terminated = False
        if self.step_counter + 1 == self.terminate_num:
            terminated = True
        self.step_counter += 1

        # observation made after modifying states. 
        observation = self._get_obs()

        if self.render_mode == "human":
            self._render_frame()

        return observation, reward, terminated, False, info

    def close(self):
        if self.window is not None:
            pygame.display.quit()
            pygame.quit()


In [45]:
env = SchedulerEnvironment(terminate_num = 50) 

In [46]:
def print_info(info):
    print("----------")
    print("current step =", info[2])
    print("-----")
    print("Average delay of first queue:", info[0][0])
    print("Average delay of second queue:", info[0][1])
    print("Average delay of best-effort queue:", info[0][2])
    print("-----")
    print("Packets removed from first queue:", info[1][0])
    print("Packets removed from second queue:", info[1][1])
    print("Packets removed from best-effort queue:", info[1][2])
    

In [47]:
def print_observation(observation):
    print("----------")
    print("First queue:", observation["queues"][0])
    print("Second queue:", observation["queues"][1])
    print("Best effort queue:", observation["queues"][2])
    

In [48]:
env.action_space.sample()

0

In [49]:
obs, info = env.reset()
print_observation(obs)
print_info(info)

----------
First queue: [1, 4, 0, 5, 1, 2, 0, 2, 1, 2]
Second queue: [1, 2, 0, 4, 2, 5, 5, 2, 4, 5]
Best effort queue: [3, 4, 0, 1, 1, 5, 1, 1, 4, 4]
----------
current step = 1
-----
Average delay of first queue: 1.8
Average delay of second queue: 3.0
Average delay of best-effort queue: 2.4
-----
Packets removed from first queue: 0
Packets removed from second queue: 0
Packets removed from best-effort queue: 0


In [51]:
terminated = False 
truncated = False
env.reset()
score = 0

while not terminated: 
    
#     random 
    action = random.randint(0, 2)
    obs, reward, terminated, truncated, info = env.step(action)
    score += reward
    print_observation(obs)
    print_info(info)
print(score)

----------
First queue: [1, 5, 5, 1, 1, 1, 5, 3, 5]
Second queue: [6, 6, 1, 5, 1, 2, 5, 6, 4, 4]
Best effort queue: [5, 3, 1, 2, 1, 1, 3, 4, 6, 6]
----------
current step = 1
-----
Average delay of first queue: 3.0
Average delay of second queue: 4.0
Average delay of best-effort queue: 3.2
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 0
----------
First queue: [6, 6, 2, 2, 2, 6, 4, 6]
Second queue: [7, 7, 2, 6, 2, 3, 6, 7, 5, 5]
Best effort queue: [6, 4, 2, 3, 2, 2, 4, 5, 7, 7]
----------
current step = 2
-----
Average delay of first queue: 4.25
Average delay of second queue: 5.0
Average delay of best-effort queue: 4.2
-----
Packets removed from first queue: 2
Packets removed from second queue: 0
Packets removed from best-effort queue: 0
----------
First queue: [7, 3, 3, 3, 7, 5, 7]
Second queue: [8, 8, 3, 7, 3, 4, 7, 8, 6, 6]
Best effort queue: [7, 5, 3, 4, 3, 3, 5, 6, 8, 8]
----------
current step = 3
-----
Avera

In [54]:
def is_empty(queue):
    if (queue == []):
        return True
    else:
        return False

In [56]:
# FIFO
def fifo_action(obs):
    first_queue = obs["queues"][0]
    second_queue = obs["queues"][1]
    best_effort_queue = obs["queues"][2]
    
    first_queue_element = None
    second_queue_element = None
    best_effort_queue_element = None
    
    if (not is_empty(first_queue)):
        first_queue_element = first_queue[0]
    
    if (not is_empty(second_queue_element)):
        second_queue_element = second_queue[0]
    
    if (not is_empty(best_effort_queue)):
        best_effort_queue_element = best_effort_queue[0]
    
    largest_delay = max(first_queue_element, second_queue_element, best_effort_queue_element)
    
    action = 0
    if (largest_delay == best_effort_queue_element):
        action = 2
    elif (largest_delay == second_queue_element):
        action = 1
    
    return action

In [57]:
terminated = False 
truncated = False
obs, info = env.reset()
score = 0
while not terminated: 
    
#     fifo
    action = fifo_action(obs)
    obs, reward, terminated, truncated, info = env.step(action)
    score += reward
    print_observation(obs)
    print_info(info)
print(score)

----------
First queue: [2, 4, 2, 1, 2, 2, 2, 4, 2]
Second queue: [3, 6, 3, 5, 5, 4, 6, 2, 1, 6]
Best effort queue: [3, 2, 4, 2, 6, 6, 5, 1, 3, 5]
----------
current step = 1
-----
Average delay of first queue: 2.3333333333333335
Average delay of second queue: 4.1
Average delay of best-effort queue: 3.7
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 0
----------
First queue: [3, 5, 3, 2, 3, 3, 3, 5, 3]
Second queue: [4, 7, 4, 6, 6, 5, 7, 3, 2, 7]
Best effort queue: [3, 5, 3, 7, 7, 6, 2, 4, 6, 0]
----------
current step = 2
-----
Average delay of first queue: 3.3333333333333335
Average delay of second queue: 5.1
Average delay of best-effort queue: 4.3
-----
Packets removed from first queue: 1
Packets removed from second queue: 0
Packets removed from best-effort queue: 1
----------
First queue: [4, 6, 4, 3, 4, 4, 4, 6, 4]
Second queue: [8, 5, 7, 7, 6, 8, 4, 3, 8]
Best effort queue: [4, 6, 4, 8, 8, 7, 3, 5, 7, 1]
----